In [1]:
import tensorflow as tf
import numpy as np
import math
import matplotlib.pyplot as plt
from tensorflow.keras import models, layers, datasets
from tensorflow.keras.layers import Dense, Flatten, Reshape, Input, InputLayer
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.initializers import RandomNormal
from tensorflow.python.keras import backend as K
from tensorflow.python.ops import math_ops
import keras
alphal0 = 1e-5
beta = 20
alphal2 = 1e-4
REG = "L2L0"
tf.keras.utils.get_custom_objects().clear()

@tf.keras.utils.register_keras_serializable(package='Custom', name='L2L0_Reg')
class L2L0_Reg(tf.keras.regularizers.Regularizer):

  def __init__(self, l0=0., beta=0, l2=0.):  # pylint: disable=redefined-outer-name
    self.l0 = K.cast_to_floatx(l0)
    self.beta = K.cast_to_floatx(beta)
    self.l2 = K.cast_to_floatx(l2)

  def __call__(self, x):
    # ones_tensor = tf.ones(x.shape)
    if not self.l2 and not self.l0:
      return K.constant(0.)
    regularization = 0.
    if self.l0:
      ones_tensor = tf.ones(x.shape)
      regularization += self.l0 * math_ops.reduce_sum(ones_tensor-math_ops.exp(-self.beta*math_ops.abs(x)))
      # regularization += self.l0 * math_ops.reduce_sum( tf.clip_by_value(self.beta * math_ops.abs(x), 0, 1)  )
    if self.l2:
      regularization += self.l2 * math_ops.reduce_sum(math_ops.square(x))
    return regularization

  def get_config(self):
    return {'l0': float(self.l0), 'beta': float(self.beta), 'l2': float(self.l2)}

  @classmethod
  def from_config(cls, config):
      l0 = float(config.pop("l0"))
      beta = float(config.pop("beta"))
      l2 = float(config.pop("l2"))
      return cls(l0=l0,beta=beta,l2=l2)


def l0reg(l0=0.0001, l2=0.001, beta=10):
  return L2L0_Reg(l0=l0, beta=beta, l2=l2)

def LeNet_NN_broken(MT):
    """
    Define LeNet 300-100-10 Dense Fully Connected
    Neural Network for MNIST multi-class classification
    """

    model = keras.Sequential()

    model.add(keras.layers.Input(shape=(28,28)))

    model.add(Flatten())

    model.add(
        Dense(units = MT, activation = None,
              kernel_initializer = tf.initializers.GlorotNormal(),
              input_shape = (784,),
              kernel_regularizer=l0reg(l0=alphal0*np.sqrt(235500/266610),l2=alphal2,beta=beta) if REG != "None" else None
             )
    )
    model.add(
          Dense(units = 300, activation = 'relu',
                kernel_initializer = tf.initializers.GlorotNormal(),
                kernel_regularizer=l0reg(l0=alphal0*np.sqrt(30100/266610),l2=alphal2,beta=beta) if REG != "None" else None     
		  )
	)

    model.add(
        Dense(units = 100, activation = 'relu',
              kernel_initializer = tf.initializers.GlorotNormal(),
              kernel_regularizer=l0reg(l0=alphal0*np.sqrt(30100/266610),l2=alphal2,beta=beta) if REG != "None" else None
             )
    )

    model.add(
        Dense(units = 10, activation = None, kernel_regularizer=l0reg(l0=alphal0*np.sqrt(1010/266610),l2=alphal2,beta=beta) if REG != "None" else None
             )
    )
    model.compile(
	
  	loss=tf.keras.losses.categorical_crossentropy,
		# optimizer='adam',
		optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0012),
		metrics=['accuracy'])
  
    return model

def extract_weights_from_model(model):
    # Extract weights from the model
    model_weights = model.get_weights()
    # Reshape and organize weights into a list of numpy arrays for each layer
    weights = []
    for layer_idx in range(0, len(model_weights), 2):
        # Get the weight matrix and the bias vector
        weight_matrix = model_weights[layer_idx]    # Shape: (input_neurons, output_neurons)
        bias_vector = model_weights[layer_idx + 1]  # Shape: (output_neurons,)
        # Combine weights and biases for each neuron
        layer_weights = []
        for neuron_idx in range(weight_matrix.shape[1]):  # Iterate through output neurons
            neuron_weights = weight_matrix[:,neuron_idx]  # Get weights for each neuron
            neuron_combined = np.append( bias_vector[neuron_idx],neuron_weights,)  # Append bias to the weights
            layer_weights.append(neuron_combined)
        
        weights.append(np.array(layer_weights))  # Append all neuron weights for this layer

    return (weights)

   

from tensorflow.keras.models import load_model

def float_to_fixed_32(number: float,decimal_bits = 28) -> np.int32:
    """
    Returns: 32-bit signed integer (usable as uint32 in hardware).
    """
    scaled = number * (2**decimal_bits) # Multiply by 2^16
    fixed = int(np.round(scaled))  # Round to nearest integer
    
    # Handle overflow (optional)
    fixed = np.int32(fixed)  # Force 32-bit signed integer
    return fixed

def to_32bit_hex(value):
    """Convert a signed integer to 32-bit hex string (SystemVerilog compatible)."""
    return f"32'h{value & 0xFFFFFFFF:08X}"  # Mask to 32 bits, uppercase, zero-padded

# Original arrays

2025-08-21 09:57:04.222989: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-21 09:57:04.226527: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-21 09:57:04.236060: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755781024.251594  144236 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755781024.256238  144236 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755781024.268873  144236 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
print(to_32bit_hex(2147483647))

32'h7FFFFFFF


In [3]:
lenet = load_model('/home/isi/Documents/Mestrado/SVD_tests/brokenLeNet.h5')
weights_lenet = extract_weights_from_model(lenet)
#generate_svd_sv_weights(weights_lenet, "lenet_weights.vhd")
lenet.summary()

2025-08-21 09:57:06.684156: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Model: "sequential_303"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_303 (Flatten)           │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1212 (Dense)              │ (None, 37)             │        29,045 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1213 (Dense)              │ (None, 300)            │        11,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1214 (Dense)              │ (None, 100)            │        30,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1215 (Dense)              │ (None, 10)             │         1,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 71,555 (279.51 KB)

 Trainable params: 71,555 (279.51 KB)

 Non-trainable params: 0 (0.00 B)

In [59]:
weights = []
biases  = []
for layer in lenet.layers:
    # Check if the layer has weights
    if len(layer.get_weights()) > 0:
        weights.append(layer.get_weights()[0])  # Extract the weight matrix
        biases.append(layer.get_weights()[1])
        print(f"Layer: {layer.name}")
        #print(f"Weights matrix shape: {weights[-1].shape}")
        #print(weights[-1])  # Display the weight matrix
        #print("="*50)
    else:
        print(f"Layer {layer.name} does not have weights.")

#%%
# Make SVD Conversion for each layer
U, S, VT, US = [],[],[],[]
for weight in weights:
    t_U , t_S , t_VT = np.linalg.svd(weight)
    t_US = np.zeros_like(t_U)
    for i in range(len(t_S)):
        t_US[:,i] = t_U[:,i]*t_S[i]
    t_US = t_US[:,:len(t_S)]
    t_Vt_send = t_VT[:len(t_S),:]
    U.append(t_U)
    S.append(t_S)
    VT.append(t_Vt_send)
    US.append(t_US)
    print(f" | |VT({len(t_VT)},{len(t_VT[0])}) |US({len(t_US)},{len(t_US[0])}) )")


Layer flatten_303 does not have weights.
Layer: dense_1212
Layer: dense_1213
Layer: dense_1214
Layer: dense_1215
 | |VT(37,37) |US(784,37) )
 | |VT(300,300) |US(37,37) )
 | |VT(100,100) |US(300,100) )
 | |VT(10,10) |US(100,10) )


In [67]:
def norm_to_fixed_32(x):
    if x > 1 :
        x = 1
        print("Number bigger than one")
    if x < -1 :
        x = -1
        print("Number smaller than minus one")
    return int((x*(2)**31) - 1) if x != 0 else 0


def write_sv_signed_array(array,template,only_numbers):
    array_string = ""
    arr_string = ""
    for i,x in enumerate(array):
        last_char = " , " if i < len(array) - 1 else ""
        arr_string+= to_32bit_hex(int(norm_to_fixed_32(x))) + last_char
    if (only_numbers == False):
        array_string += f"logic signed [31:0] "+ template + f"[{len(array)}] = '{{{arr_string}}};\n"
    else :
        array_string += arr_string
    return array_string


def write_sv_signed_matrix(matrix,template):
    
    matrix_string = "\nlogic signed [31:0] "+template+f" [{matrix.shape[0]}][{matrix.shape[1]}]"+" ='{\n"

    for i,arr in enumerate(matrix):
        last_char = " , " if i < matrix.shape[0] - 1 else ""
        matrix_string+= "{" + write_sv_signed_array(arr,'',True) + "}" +last_char + "\n"
    matrix_string+='};\n'
    
    return matrix_string


def generate_svd_sv_weights(VT,US,biases, file_name):
    sv_template = """

package model_weights;

{weight_constants}

endpackage
"""
    weight_constants = ""
    
    for i,v in enumerate(VT):
        weight_constants+=write_sv_signed_matrix(v,f"VT_{i}")
        print(v.shape)
    
    
    for i,us in enumerate(US):
        us_np = np.array(us)
        weight_constants+=write_sv_signed_matrix(us_np,f"US_{i}")
        print(us_np.shape)


    for i,b in enumerate(biases):
        weight_constants+=write_sv_signed_array(b,f"bias_{i}",False)
        
    sv_code = sv_template.format(
        weight_constants=weight_constants
    )
    with open(file_name, "w") as sv_file:
        sv_file.write(sv_code)

# Example usage:
# Assuming you have extracted weights from your neural network model
# Here, weights is a list of numpy arrays representing the weights for each layer

In [68]:
generate_svd_sv_weights(VT,US,biases,"svd_values.sv")

(37, 37)
(37, 300)
(100, 100)
(10, 10)
Number bigger than one
Number smaller than minus one
Number bigger than one
Number smaller than minus one
Number smaller than minus one
Number smaller than minus one
(784, 37)
(37, 37)
Number smaller than minus one
Number smaller than minus one
Number smaller than minus one
Number smaller than minus one
Number smaller than minus one
Number bigger than one
Number smaller than minus one
Number smaller than minus one
Number smaller than minus one
Number bigger than one
Number bigger than one
Number smaller than minus one
Number bigger than one
Number smaller than minus one
Number smaller than minus one
Number smaller than minus one
Number bigger than one
Number bigger than one
Number bigger than one
Number smaller than minus one
Number bigger than one
Number smaller than minus one
Number bigger than one
(300, 100)
Number bigger than one
Number bigger than one
Number smaller than minus one
Number smaller than minus one
Number smaller than minus one
Nu

In [69]:

def normalize_with_numpy(nested_list):

    result = []
    
    for inner_list in nested_list:   
        arr = np.array(inner_list)
        max_val = abs(arr).max()
        
        
        if max_val == 0:
            normalized = np.zeros_like(arr, dtype=float)
        else:
            normalized = arr/(max_val*1.000001)
        result.append(normalized)
    
    return result , 1/(max_val*1.000001)


In [71]:
#Normalizar cada matriz de pesos


VT_n ,alpha= normalize_with_numpy(VT)
US_n ,beta= normalize_with_numpy(US)
#print(alpha,beta)
biases_n = []
for bias in biases:
    biases_n.append(bias*alpha*beta)

#print(VT[0].max(),VT[0].min())
#print(VT_n[0][1,1],VT[0][1,1])
generate_svd_sv_weights(VT_n,US_n,biases_n,"svd_values_norm.sv")
#print(US[1])

(37, 37)
(37, 300)
(100, 100)
(10, 10)
(784, 37)
(37, 37)
(300, 100)
(100, 10)


In [ ]:
#Normalizar por um fator comum entre todas as variaveis
def normalize_general(VT,US,biases):
    result_VT,result_US,result_biases = []
    
    for inner_list in nested_list:   
        arr = np.array(inner_list)
        max_val = abs(arr).max()
        
        
        if max_val == 0:
            normalized = np.zeros_like(arr, dtype=float)
        else:
            normalized = arr/(max_val*1.000001)
        result.append(normalized)
    
    return result , 1/(max_val*1.000001)

VT_n ,alpha= normalize_with_numpy(VT)
US_n ,beta= normalize_with_numpy(US)
#print(alpha,beta)
biases_n = []
for bias in biases:
    biases_n.append(bias*alpha*beta)

#print(VT[0].max(),VT[0].min())
#print(VT_n[0][1,1],VT[0][1,1])
generate_svd_sv_weights(VT_n,US_n,biases_n,"svd_values_norm.sv")
#print(US[1])

In [64]:
print(biases[1])


[-0.26857463  0.11653159 -0.03921171  0.3379265  -0.11215492 -0.05069784
 -0.01574498 -0.07947755 -0.04679665 -0.13859864 -0.17103831  0.11341953
  0.29205248 -0.01493471 -0.13107313  0.01520232 -0.0174751   0.05377271
 -0.33874756 -0.00943521 -0.01615655 -0.08488752  0.33888295  0.05655032
  0.19301003 -0.01016331 -0.03299653  0.02159429 -0.01120275  0.41458073
 -0.0205394   0.09969426 -0.20669778 -0.07146092 -0.0085628   0.02664064
  0.19696796 -0.21724004 -0.07529428  0.00544734 -0.19035766 -0.07074089
 -0.01662887  0.04591956  0.31902382  0.36379084 -0.00366823 -0.00825527
  0.08657182 -0.06879878 -0.04655615 -0.02881224 -0.0990693   0.15883671
 -0.06439045 -0.00712355 -0.03220049 -0.04481666 -0.00914741 -0.50608665
 -0.03217473 -0.02538256 -0.04552635 -0.01192292  0.15831415 -0.04273221
 -0.10854637  0.0268755  -0.01593244 -0.3005913  -0.08288822 -0.24838582
 -0.02632396 -0.01005456 -0.01806499 -0.3336969  -0.09432822 -0.01424101
  0.18302883 -0.1520102  -0.10035869  0.2509785  -0

In [65]:
print(biases_n[1])

[-0.23558792  0.10221902 -0.03439568  0.29642195 -0.0983799  -0.04447107
 -0.01381116 -0.06971601 -0.04104903 -0.12157577 -0.15003116  0.09948918
  0.2561822  -0.01310041 -0.11497456  0.01333516 -0.01532879  0.04716827
 -0.29714215 -0.00827637 -0.01417218 -0.07446152  0.2972609   0.04960473
  0.16930428 -0.00891504 -0.02894386  0.01894205 -0.00982681  0.36366138
 -0.01801672  0.08744968 -0.18131088 -0.06268399 -0.0075111   0.0233686
  0.1727761  -0.19055833 -0.06604654  0.00477829 -0.16697767 -0.0620524
 -0.01458649  0.04027966  0.2798409   0.31910956 -0.0032177  -0.00724135
  0.07593895 -0.06034882 -0.04083806 -0.02527348 -0.08690147  0.13932817
 -0.05648193 -0.00624862 -0.02824559 -0.03931222 -0.00802392 -0.44392842
 -0.02822299 -0.02226504 -0.03993474 -0.01045854  0.13886979 -0.03748379
 -0.09521455  0.02357462 -0.01397559 -0.26367226 -0.07270779 -0.21787874
 -0.02309082 -0.00881965 -0.01584622 -0.2927118  -0.08274271 -0.01249191
  0.16054899 -0.1333401  -0.08803251  0.22015299 -0.0

In [25]:
from tensorflow.keras.datasets import mnist


In [ ]:
#Compile lenet model
lenet.compile(optimizer='adam',loss='mean_squared_error')

#Load MNIST Dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize pixel values (0-255 → 0-1)
train_images = train_images / 255.0
test_images = test_images / 255.0

#Select One image
img_processed = train_images[0].reshape(1, 28, 28, 1)
img_processed_tb = img_processed.reshape(-1,1)
print("Training data shape:", train_images.shape, img_processed.shape)  # (60000, 28, 28)
print("Test data shape:", test_images.shape , test_labels.shape)       # (10000, 28, 28)
#Predict Output
result = lenet.predict(img_processed)

#Convert to .csv, read and store in array on testbench
print(img_processed_tb.shape)
#X_tb = list(np.zeros_like(img_processed_tb))
X_tb = []
for i,x in enumerate(img_processed_tb):
    X_tb.append(int(norm_to_fixed_32(x)))

print(type(X_tb),(X_tb.reverse()))
np.savetxt('mnist_pixel_data.csv', 
           X_tb[::-1], 
           fmt='%X',          # Decimal format
           delimiter=',',     # Comma-separated
           header='',         # No header
           comments='')       # No comment characters

X_tb_layer2 = []

for i,x in enumerate(img_processed_tb):
    X_tb.append(int(norm_to_fixed_32(x)))

Training data shape: (60000, 28, 28) (1, 28, 28, 1)
Test data shape: (10000, 28, 28) (10000,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
(784, 1)
<class 'list'> None


/tmp/ipykernel_144236/4100426010.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return int((x*(2)**31) - 1) if x != 0 else 0


In [31]:
print(result)
lenet.summary()

[[2.6715745e-15 4.7046100e-10 5.9327407e-13 2.1920911e-04 8.9815265e-22
  9.9978083e-01 1.9406104e-18 5.7118121e-12 7.6158898e-16 2.4271533e-11]]


Model: "sequential_303"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_303 (Flatten)           │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1212 (Dense)              │ (None, 37)             │        29,045 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1213 (Dense)              │ (None, 300)            │        11,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1214 (Dense)              │ (None, 100)            │        30,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1215 (Dense)              │ (None, 10)             │         1,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 71,555 (279.51 KB)

 Trainable params: 71,555 (279.51 KB)

 Non-trainable params: 0 (0.00 B)

In [32]:
import keras

In [27]:

w0 = lenet.layers[1].get_weights()
w1 = lenet.layers[2].get_weights()
modelflayer = keras.Sequential()

modelflayer.add(keras.layers.Input(shape=(28,28)))
modelflayer.add(Flatten())
modelflayer.add(
    Dense(units = 37, activation = None,
          kernel_initializer = tf.initializers.GlorotNormal(),
          input_shape = (784,),
          kernel_regularizer=l0reg(l0=alphal0*np.sqrt(235500/266610),l2=alphal2,beta=beta) if REG != "None" else None
         )

)

modelflayer.layers[1].set_weights(w0)
test_1 = modelflayer.predict(img_processed)
modelflayer.add(
    Dense(units = 300, activation = "relu",
          kernel_initializer = tf.initializers.GlorotNormal(),
          input_shape = (37,),
          kernel_regularizer=l0reg(l0=alphal0*np.sqrt(235500/266610),l2=alphal2,beta=beta) if REG != "None" else None
         )
)
#test_2 = modelflayer.predict(img_processed)
#modelflayer.layers[2].set_weights(w1)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


/home/isi/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
def signed_to_unsigned(n, bits=32):
    mask = (1 << bits) - 1  # Create bitmask
    return n & mask


X_tb = []
test_1_n,_ = normalize_with_numpy(test_1)
print(test_1[0])
print(test_1_n[0])

for i,x in enumerate(test_1_n[0]):
    X_tb.append(signed_to_unsigned(int(norm_to_fixed_32(x))))

print(type(X_tb))


np.savetxt('layer_1_exit.csv', 
           X_tb, 
           fmt='%X',          # Decimal format
           delimiter=',',     # Comma-separated
           header='',         # No header
           comments='')       # No comment characters
print(X_tb)

[ 2.3235943   5.337052    1.686502   -2.9129078   2.8752513   1.7589655
  3.8717318   0.64927065  3.090874   -0.36634356 -0.42473787 -1.9393029
 -3.5138793  -0.02679098  1.678901    2.8468308  -1.1673747   4.7979307
 -2.5374403   3.1928668   1.5989801  -2.4882338   4.5781097  -0.1746226
  7.128281   -1.8107544  -1.1275764   1.8484052   0.94436884 -1.4971018
 -3.0385048   0.09838134  3.43033     0.01653632  2.7478933  -1.1405938
 -1.4388838 ]
[ 0.3259681   0.74871445  0.23659286 -0.4086406   0.40335792  0.24675849
  0.5431503   0.09108368  0.43360677 -0.05139292 -0.05958484 -0.27205732
 -0.49294856 -0.0037584   0.23552655  0.3993709  -0.1637665   0.6730832
 -0.3559677   0.44791496  0.22431475 -0.34906474  0.64224535 -0.02449713
  0.99999905 -0.25402376 -0.15818334  0.25930566  0.13248186 -0.21002263
 -0.4262601   0.01380154  0.48122776  0.00231982  0.38549137 -0.1600095
 -0.20185545]
<class 'list'>
[700011136, 1607852032, 508079296, 3417418304, 866204544, 529909824, 1166406400, 19560070

In [156]:
print(test_2)

[[0.         0.         0.16752437 0.         0.15051463 0.
  0.         0.         0.1034645  0.53264534 0.77050054 0.
  0.         1.3102565  0.         1.6889466  1.4458104  1.5346419
  1.8858335  0.8408802  0.         0.         0.         0.4096307
  0.         0.         0.10526977 0.         0.         0.
  0.         1.1062775  0.         0.         0.7425416  0.27777687
  0.48638934 0.         1.712933   0.         1.2175151  0.
  1.0655732  0.         0.         0.         1.7420594  2.77586
  0.01395621 0.         0.         0.         0.         0.
  0.         0.         0.         0.16611505 0.         0.
  0.         1.486958   0.         0.         0.         0.22953513
  0.         0.         0.7870524  0.63481724 0.         1.6979332
  0.         0.50029165 0.         0.01502359 1.6738992  0.
  0.11210147 2.0073228  0.9874821  0.         0.         1.2213451
  0.         0.5637371  0.5196267  0.         0.         1.3976003
  0.60139745 0.5958304  0.         1.9593477

In [36]:
lenet_n = LeNet_NN_broken(37)

for layer_n, layer in zip(lenet_n.layers,lenet.layers):
    layer_n.set_weights(layer.get_weights())

In [37]:
result_n = lenet_n.predict(img_processed)
print(result_n)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
[[-14.616201   -2.5374002  -9.213213   10.514422  -29.521795   18.939688
  -21.84362    -6.9485784 -15.871219   -5.5018096]]
[[2.6715745e-15 4.7046100e-10 5.9327407e-13 2.1920911e-04 8.9815265e-22
  9.9978083e-01 1.9406104e-18 5.7118121e-12 7.6158898e-16 2.4271533e-11]]


In [38]:
VT[0]

array([[-1.0000000e+00,  5.2779900e-08, -3.5785011e-08, ...,
        -1.4185846e-09,  1.4923333e-09,  9.1320718e-10],
       [ 5.2779900e-08,  1.0000000e+00, -7.0199566e-09, ...,
        -3.2623835e-09,  3.8577510e-09, -1.0067778e-09],
       [-3.5785007e-08,  7.0199584e-09,  1.0000000e+00, ...,
        -4.8951876e-10, -1.8148121e-09, -1.0470478e-09],
       ...,
       [ 1.4185849e-09, -3.2623855e-09, -4.8951754e-10, ...,
        -1.0000000e+00,  4.6635111e-07,  6.8266573e-08],
       [ 1.4923325e-09, -3.8577488e-09,  1.8148126e-09, ...,
         4.6635111e-07,  1.0000000e+00,  1.9781384e-08],
       [-9.1320695e-10, -1.0067783e-09, -1.0470474e-09, ...,
        -6.8266559e-08,  1.9781417e-08, -1.0000000e+00]], dtype=float32)

In [39]:
import numpy as np

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

print(softmax(result_n))

[[2.6715745e-15 4.7046095e-10 5.9327407e-13 2.1920909e-04 8.9815265e-22
  9.9978083e-01 1.9406104e-18 5.7118121e-12 7.6158898e-16 2.4271531e-11]]


In [47]:
#Recreate Lenet SVD using weights until layer 2
def relu(x):
    return x if x > 0 else 0

print((len(w0[0])))

X = img_processed_tb.T
H = np.array(w0[0])
H2 = np.array(w1[0])
print(X.shape,H.shape,H2.shape)
Y =  (X @ H)  #+ w0[1]
Y2 = (Y @ H2) #+ w1[1]
Y2_relu = []

for y in Y2:    
    Y2_relu.append(list(map(relu,(y))))
U , S , VT = np.linalg.svd(H)
US = np.zeros_like(U)

for i in range(len(S)):
    US[:,i] = U[:,i]*S[i]
US = US[:,:len(S)]


U2 , S2 , VT2 = np.linalg.svd(H2)
US2 = np.zeros_like(U2)

for i in range(len(S2)):
    US2[:,i] = U2[:,i]*S2[i]
US2 = US2[:,:len(S2)]


Y_test = w1[1].reshape(1,300)

Y_test0 = w0[1].reshape(1,37)

data_df = []
data_df0 = []
for i in range(len(S)):
    data_info0 = []
    data_info0.append(i)
    USX = X @ US[:,i]
    data_info0.append(USX[0])
    temp = np.outer(USX , VT[i,:])
    #for t in temp[0,:]:
    #    data_info0.append(t)
    Y_test0 += temp
    #for y in Y_test[0,:]:
    #    data_info.append(y)
    data_df0.append(data_info0)

for i in range(len(S2)):
    data_info = []
    data_info.append(i)
    USX = Y @ US2[:,i]
    data_info.append(USX[0])
    temp = np.outer(USX , VT2[i,:])
    #for t in temp[0,:]:
    #    data_info.append(t)
    Y_test += temp
    #for y in Y_test[0,:]:
    #    data_info.append(y)
    data_df.append(data_info)

print(f"{X.shape = } | {U.shape =} | {US.shape = } |{VT.shape = } |{Y.shape = } |{H.shape = } |")




784
(1, 784) (784, 37) (37, 300)
X.shape = (1, 784) | U.shape =(784, 784) | US.shape = (784, 37) |VT.shape = (37, 37) |Y.shape = (1, 37) |H.shape = (784, 37) |


In [50]:
print(Y2)
print(VT)

[[ 2.52945785e+00 -4.53769199e-01  1.38171573e-04  4.24034965e-02
   1.29348138e-03 -1.43056902e-01  1.51170273e-03  1.33271594e-03
   1.55642194e-04  1.88476766e-04 -1.44795607e-01 -5.75763794e-01
  -4.86529845e-01  6.03630599e-04 -1.74783192e-03  2.27835264e+00
  -1.18108444e-04  3.10362545e-01  2.14822861e-01  1.57141617e-03
   9.29150289e-06  8.83605162e-02 -3.89709095e-01  5.60960400e-01
   6.35608794e-01 -8.77901925e-04 -3.77040458e-01 -3.46221079e-01
   4.13624398e-04 -3.99238165e+00 -5.22401440e-04  1.01780069e+00
   1.18859818e-03 -8.52173646e-04 -6.64843645e-04  2.62561559e+00
  -1.29588632e+00  7.67226047e-01 -8.17803238e-04 -3.35689945e+00
  -1.45140684e+00 -3.42150486e-04  8.48130235e-05 -2.02045572e-01
  -5.02799300e+00 -7.77138987e-01 -1.10997527e+00 -1.18553140e-03
   1.12345666e+00  2.70650362e-01 -1.72376979e-01  1.22648324e-03
   1.51946467e-01  1.35929469e+00  1.41533441e+00 -1.12258673e-04
   5.15077560e-03  2.21563095e-04 -1.00389830e-03  6.32411066e-01
  -8.05523

In [95]:
import pandas as pd
df = pd.DataFrame(data_df0)
df

,0,1
0,0,-2.323594
1,1,5.337052
2,2,1.686502
3,3,-2.912908
4,4,-2.875251
5,5,-1.758965
6,6,-3.871732
7,7,-0.649270
8,8,-3.090874
9,9,-0.366343


In [36]:
import pandas as pd
df = pd.DataFrame(data_df)
df

,0,1
0,0,-5.274517e+02
1,1,-5.123261e-07
2,2,-1.802420e-06
3,3,1.219638e-06
4,4,-1.570096e-06
5,5,-2.106216e-06
6,6,-9.621696e-07
7,7,-2.498457e-07
8,8,-1.970198e-06
9,9,4.905303e-06


In [18]:
print (w1[1])

[-0.26857463  0.11653159 -0.03921171  0.3379265  -0.11215492 -0.05069784
 -0.01574498 -0.07947755 -0.04679665 -0.13859864 -0.17103831  0.11341953
  0.29205248 -0.01493471 -0.13107313  0.01520232 -0.0174751   0.05377271
 -0.33874756 -0.00943521 -0.01615655 -0.08488752  0.33888295  0.05655032
  0.19301003 -0.01016331 -0.03299653  0.02159429 -0.01120275  0.41458073
 -0.0205394   0.09969426 -0.20669778 -0.07146092 -0.0085628   0.02664064
  0.19696796 -0.21724004 -0.07529428  0.00544734 -0.19035766 -0.07074089
 -0.01662887  0.04591956  0.31902382  0.36379084 -0.00366823 -0.00825527
  0.08657182 -0.06879878 -0.04655615 -0.02881224 -0.0990693   0.15883671
 -0.06439045 -0.00712355 -0.03220049 -0.04481666 -0.00914741 -0.50608665
 -0.03217473 -0.02538256 -0.04552635 -0.01192292  0.15831415 -0.04273221
 -0.10854637  0.0268755  -0.01593244 -0.3005913  -0.08288822 -0.24838582
 -0.02632396 -0.01005456 -0.01806499 -0.3336969  -0.09432822 -0.01424101
  0.18302883 -0.1520102  -0.10035869  0.2509785  -0

In [19]:
print(Y)

[[ 2.32359399  5.33705205  1.68650199 -2.91290789  2.87525063  1.75896574
   3.87173207  0.64927052  3.09087423 -0.36634363 -0.42473805 -1.93930306
  -3.51387976 -0.02679064  1.6789015   2.84683081 -1.16737453  4.7979307
  -2.53744028  3.19286705  1.59898031 -2.48823397  4.57810914 -0.17462273
   7.12828105 -1.81075459 -1.12757663  1.84840495  0.94436889 -1.49710182
  -3.03850454  0.09838142  3.43033007  0.01653623  2.74789343 -1.14059371
  -1.43888371]]


In [20]:
print(w0[1])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
